# Hjelp jeg prøvde å bruke Fully Homomorphic Encryption (FHE)

* Hva er Homomorphic Encryption
  * Eksempler
* Hva er Fully Homomorphic Encryptions?
  * Eksempler
* Mitt forsøk på å bruke FHE

### Hva er kryptering og hva brukes det til?
* Lagre data
* Sende data

* Behandle data?

![Screenshot 2023-02-22 164309.png](Screenshot_2023-02-22_164309.png)
>the encryption function
used
will
permit the
computer system to
operate
on
the data without decrypting
it.

I 1978 lurte 3 smarte menn fra MIT på hvordan en fiktiv bank kunne bruke en server (time-shared service) til å kalkulere sensitive data fra sine låntakere uten å gi fra seg noe informasjon. Et av forslagene var å bruke homomorfisk kryptering.

Dette var kun teoretisk i 30 år før Gentry laget det først FHE rammeverket i 2009. I 2011 kom Microsoft ut med et paper “Can Homomorphic Encryption be Practical?”. I 2018 kom Microsoft ut med en implementasjon kalt Simple Encrypted Arithmetic Library (SEAL). Og i 2022 kom Edge med Password Monitor som bruker FHE til å sjekke om passordet ditt finnes i lekkede passord-databaser. (Chrome kom kort tid etterpå med en hash-basert løsning).


# Hva er Homomorphic Encryption?
* Anna har en melding $m$.
* Anna krypterer $m$. $\text{enc}(m)$.
* Anna sender $\text{enc}(m)$ til server.
* Serveren gjør en funksjon $f()$ på den krypterte meldingen $\text{enc}(f(m))$.
* Serveren sender krypterte resultatet $\text{enc}(f(m))$ tilbake.
* Anna kan dekryptere svaret. $\text{dec}((\text{enc}(f(m))) = f(m)$.

### RSA
1. Bob selects two large prime $p<<0$ and $q<<0$ and calclate $n=p \times q$
2. Bob calculate $\phi(n) = (p-1)(q-1)$. Choose $e$ such that $\gcd(e, \phi(n))=1$.
3. Bob calculate private key $d$ to be the modular inverse of $e$, in other words $de \equiv 1 (\mod \phi(n))$.
4. Bob sends the public key $p_k = n,p$
5. Anna encrypts message $m$ to ciphertext $c \equiv m^e (\mod n)$. And sends it.
6. Bob computes the message $c^d \equiv m (\mod n)$

Før FHE RSA

In [8]:
from math import gcd
# Bobs computer
p, q, e = 11, 17, 3 # pretend that p and 1 are large (4096 bits)
n = p * q #187
print(f'{n=}')
phi_n = (p-1)*(q-1)
print(f'{gcd(e, phi_n)}')
d = pow(e, -1, mod = phi_n)

# Annas computer
message_sent = 42
cipher = message_sent**e % n
print(f'{cipher=}')

# Bobs computer
message_recieved = cipher**d % n
print(f'{message_recieved=}, {message_recieved==message_sent=}')


n=187
1
cipher=36
message_recieved=42, message_recieved==message_sent=True



### Multiplicative homomorphism in RSA

In [2]:
from math import gcd
# Bobs computer
# p, q, e = 11, 17, 3
#p, q = 101, 103
# e = 7
n = p * q
print(f'{n=}')
phi_n = (p-1)*(q-1)
print(f'{gcd(e, phi_n)=}')
d = pow(e, -1, mod = phi_n)

# Annas computer
message_sent_j = 10
message_sent_k = 42
message_product = message_sent_j * message_sent_k # Only for verification
cipher_j = message_sent_j**e % n
cipher_k = message_sent_k**e % n
print(f'{(cipher_j,cipher_k)=}')

#Homomorphic calculation
cipher_product = cipher_k * cipher_j

# Bobs computer
message_recieved = cipher_product**d % n
print(f'{(message_recieved, message_recieved==message_product, message_recieved==message_product%n)=}')

n=10403
gcd(e, phi_n)=1
(cipher_j,cipher_k)=(2717, 295)
(message_recieved, message_recieved==message_product, message_recieved==message_product%n)=(420, True, True)


### Takebacks
Ikke evig multiplikasjoner, `ciper_product`$ \equiv  m_j \times m_j \bmod {n}$

Dette kalles partially Homomorphic schemes. Kan enten pluss eller gange. Ikke evig multiplikasjoner, ciper_product ≡𝑚𝑗×𝑚𝑗mod𝑛

## Paillier

### Generate keys
$$
\begin{align}
\mathbb{Z}_n &= \{0,1,2, \ldots, n-1\} \\
p,q &\in \mathbb{N}_{p(rime)} | \gcd ( pq, (p−1)(q−1))=1 \\
n &=pq\\
\lambda &= \text{lcm} (p-1,q-1) \\
g   &\in_R \mathbb{Z}_{n^2}^* \\
L(x)& = {{x-1}\over n} \\
\mu &=(L(g^{\lambda }{\bmod \; n}^{2}))^{{-1}}{\bmod \; n}
b \\
\text{public key} =  k_{pub} &= (n, g)  \\
\text{private key} = k_{priv} &= (\lambda, \mu)  \\
\end{align}
$$

Extended Euclidean Algorithm for å finne invers

$$
\text{\{public key, private_key\}} =  \{k_{pub},k_{priv}\}  =  \{(n, g),(\lambda, \mu)\}
$$
### Encrypt one message
$$
\begin{align}
\text{message} &= m \in \mathbb{Z}_n \\
\text{random seed} &= r \in_R \mathbb{N} | \gcd(r,n)=1 \\
\text{ciphertext} &=c =\mathcal{E}(m) = g^m \times r^n &\bmod\; n^2 \\
\end{align}
$$

### Decrypt
$$
\begin{align}
L(x)& = {{x-1}\over n} \\
m &=L(c^{\lambda} \bmod \; n^2) \times \mu \bmod \; n
\end{align}
$$

Legg merke til at man multipliserer $g^m$ og $r^n$

$$
\text{\{public key, private_key\}} =  \{k_{pub},k_{priv}\}  =  \{(n, g),(\lambda, \mu)\}
$$
### Encrypt two messages
$$
\begin{align}
\mathcal{E}(m_1) &= g^{m_1} r_1^n&\bmod \;n^2 \\
\mathcal{E}(m_2) &= g^{m_2} r_2^n&\bmod \;n^2 \\
\end{align}
$$

### Homomorphic addition of two ciphertexts
$$
\begin{align}
\mathcal{E}(m_1) \times \mathcal{E}(m_2) &= (g^{m_1} r_1^n)\times(g^{m_2} r_2^n) \bmod \;n^2 \\
&= g^{m_1 + m_2} (r_1 \times r_2)^n \bmod\; n^2 &\; r_x = r_1\times r_2 \\
&= \mathcal{E}(m_1 + m_2)\\
\end{align}
$$

Man multipliserer krypterte outputten for å addere message.

### Homomorphic mulitplication of one ciphertext and non-encrypted constant  $k$
$$
\begin{align}
\mathcal{E}(m)            &= g^m r^n\\
\mathcal{E}(m) \times g^k &= g^m r^n \times g^k\\
                          &= g^{m+k} r^n\\
\end{align}
$$

In [9]:
from phe import paillier
public_key, private_key = paillier.generate_paillier_keypair()
messages = [33, 66]
cipher_texts = [public_key.encrypt(x) for x in messages]

# Ciphertext addition
cipher_texts.append(cipher_texts[0]+cipher_texts[1])
# Ciphertext multiplication will fail
# cipher_texts.append(cipher_texts[0]*cipher_texts[1])

# Non encrypted constant
cipher_texts.append(cipher_texts[0]+10) # 33 + 10
cipher_texts.append(cipher_texts[0]*10) # 33 * 10

decrypted = [private_key.decrypt(x) for x in cipher_texts]
print(f'{decrypted=}')

decrypted=[33, 66, 99, 43, 330]


Notater:


# Hva kan homomorphism brukes til?
* Anonym avstemning
* Anonym treningsdata til K.I.
* Kan kanskje *ikke* brukes der du *ikke* vil ha *mallebility*.
    * Noen hacker din krypterte IOU liste og ganger alle beløp med 2.
* ?? Kanskje ikke så mye mer...?
* ...Men hva om vi fikk til både $\times$ OG $+$ ???

# $\{ \times, + \} \rightarrow \{\wedge, \vee, \oplus, \; \neg \} \Leftrightarrow \{\&,|,\text{^},!\} $

$$
\begin{align}
a,b &\in\{0,1\}\\
a\oplus b & \Leftrightarrow a+b &\bmod 2 \\
a\wedge b & \Leftrightarrow a\times b \\
a \vee b & \Leftrightarrow a + b + (a \times b) &\bmod 2
\end{align}
$$

| $a$ | $b$ |$ a\oplus b$|$a \wedge b$|$a \vee b $|
|:--|:--|:--|:--|:--|
|$a$|$b$|$a+b\bmod 2$|$a\times b$| $a + b + (a \times b) \bmod 2$|
| 0 | 0 |$0+0=0$     |$0 \times 0 = 0$|$0+0+(0 \times0) = 0$|
| 0 | 1 |$0+1=1$     |$0 \times 1 = 0$|$0+1+(0\times 1) =1$|
| 1 | 0 |$1+0=1$     |$1 \times 0 = 0$|$1+0+(1\times 0) =1$|
| 1 | 1 |$1+1= 2 \equiv0\bmod2$|$1 \times 1 = 1$|$1+1+(1\times 1)=3\equiv1 \bmod 2$|



# FHE **F**ully **H**omomorphic **E**ncryption

### Historie (ref wiki)
* 1978
    * Teorisert
* 1st gen
    * Gentry lagde et *somewhoat homomorphic* krypterings-scheme. Dette kunne plausibilt bli FHE ved å *bootstrappe*(2009)- *Bootstrapping er når man kan implementere dekrypteringsalgoritmen i schemet.*
    * Fully Homomorphic Encryption over the Integers (2010) https://eprint.iacr.org/2009/616
* 2nd gen
    * **BGV**(2011), LTV(2012), **BFV**(2012), GHS(2012), BLLN(2013)
* 3rd gen
    * FHEW(2014)
    * TFHE(2016) Asiacrypt 2016+2017
        * Finnes et rust-implementasjon som ser ut til å være nokså godt vedlikeholdt.
        * > decrease the running time of their [FHEW] bootstrapping from 690ms to 13ms single core, using 16MB bootstrapping key instead of 1GB
* 4th gen
    * **CKKS**(2016)
        * Homomorphic Encryption for Arithmetic of Approximate Numbers (HaaN) 2016, ASIACRYPT 2017
        * In applications such as summing up encrypted real numbers, evaluating machine learning models on encrypted data, or computing distances of encrypted locations CKKS is going to be by far the best choice. (SEAL)
        * In short, our encoding function is given by:
        * ![Screenshot](CKKS_enc.png)
        * > ...we show that our scheme can be applied to the efficient evaluation of transcendental functions such as multiplicative inverse, exponential function, logistic function and discrete Fourier transform.
        * > The primary open problem is finding way to convert our scheme to a fully homomorphic scheme using bootstrapping.
* 2018
    * Microsoft Simple Encrypted Arithmetic Library (SEAL)
        * BGV BFV og CKKS


### Pros and cons

**Pros**
* Kan jobbe med kryptert data!
* Quantum-safe

**Cons**
* Tregt med en stor T.
    * 1 000 -> 1 000 000 saktere og bootstrapping kan ta opp itl flere minutt.
* Krever mye minne.
* Vanskelig å implementere (Iallefall for meg)

Såkalt leveled scheme. Kan kun brukes til alle levlene er brukt opp. Bootstrapping
* Når du har nådd syøt-budsjettet ditt, kan du ta imot tallet ditt, decryptere det, og sende det inn igjen krypter.
* Bootstrapping: Å dekryptere kryptert data kryptert.

## Eksempler

### Edit-distance
Microsoft prøvde i 2015 å se på edit-disctance på DNA. Her brukes et 4 tegns alfabet, **A**denine , **T**hymine, **G**uanine, **C**ytosine.

https://www.microsoft.com/en-us/research/publication/homomorphic-computation-of-edit-distance/

|(n, m)| Depth | Ring Mod $\Phi_d$ |Key Enc|Total|Amortized|
|-----------|---|-------|-------|-------|-------|
|(1,1)|1|d=4369|256|1.4761s|0.1118s|0.0693s|0.0003s|
|(2,2)|2|d=4369|256|1.8358s|0.2844s|0.2532s|0.0009s|
|(3,3)|8|d=8191|630|7.0162s|1.7117s|34.3091s|0.0544s|
|(4,4)|9|d=8191|630|7.4489s|2.4154s|67.5116s|0.1071s|
|(6,6)|16|d=13981|600|16.1076s|9.9498s|26min|33s|2.6555s|
|(8,8)|19|d=15709|682|27.5454s|16.4524s|4h 50min|25.4366s|
|(50,50)|||||~1 day|

> Currently we could not implement our algorithm for larger parameters [than (8,8)] due to large memory requirements

### FHEAAS
Sør Koreanske ncloud.com gjør det mulig å utføre FHE på data for oss dødelige. Bruker Homomorphic Encryption for Arithmetic of Approximate Numbers (HEaaN) aka Cheon-Kim-Kim-Song CKKS.
![heaan example](heaan-example_extask2_vpc_en.png)


### Microsoft Edge - Password manager
![Edge-password-monitor](Edge-password-monitor.png)
Private Set Intersection (PSI) refers to a functionality where two parties, each holding a private set of items, can check which items they have in common without revealing anything else to each other.

## Dypdykk i CKKS

![Cryptotree_diagrams-2.svg](https://blog.openmined.org/content/images/2020/08/Cryptotree_diagrams-2.svg)
Message $m$ vector (vector med floats) blir encodet til med en *Scaling factor* $\Delta$ til Plaintext .
Plaintext blir kryptert og får lengden $\log Q$ bits

### Multiplisering

* Message $m$ vector (vector med floats) blir encodet til med en *Scaling factor* $\Delta$ til Plaintext .
* Plaintext blir kryptert og får lengden $\log Q$ bits
* $Q = q_0 \times \Delta ^ \ell $
    * $ q_0$: Base modulus
    * $ Q_l = q_0 \times \Delta ^\ell$: Ciphertext modulus
* Level
    * Jeg ser på level $\ell$ som liv i zelda, og når man har 0 liv igjen, kan man ikke f.eks gjøre flere multiplikasjoner.
* Overordnet multiplikasjon.
    * Multiplikasjon: $(ct_a, \ell, \Delta)\times(ct_b,\ell,\Delta) \rightarrow (ct_{mul}, \ell, \Delta^2)$. Scale øker.
    * Relinearization: $(ct_{mul},\ell,\Delta^2)(ct'_{mul},\ell, \Delta^2)$ Relinearization gjør selve cipherteksten mindre.
    * Rescale: $(ct'_{mul},\ell,\Delta^2)\rightarrow(ct''_mul,\ell-1,\Delta)$

### Multipliseringsrekkefølge $((x\times y) \times z \times w$
![xy*z*y](xy_z_w.png)

### Multipliseringsrekkefølge $(x\times y) \times (z \times w)$

![xy_zy.png](xy_zw.png)

## Mitt forsøk på å regne ut $\pi x^3 + 0.4x + 1$

In [11]:
from seal import (
    EncryptionParameters,
    scheme_type,
    CoeffModulus,
    PlainModulus,
    SEALContext,
    KeyGenerator,
    Encryptor,
    Evaluator,
    Decryptor,
    BatchEncoder,
    CKKSEncoder,
    Ciphertext,
    Plaintext,
)

from math import log2


def print_vector(vector, rows, cols):
    print('[')
    for r in range(rows):
        print(*vector[r*cols:(r+1)*cols], sep='\t')
    print(']')

# In this example we demonstrate evaluating a polynomial function

#     PI*x^3 + 0.4*x + 1

# on encrypted floating-point input data x for a set of 4096 equidistant points
# in the interval [0, 1]. This example demonstrates many of the main features
# of the CKKS scheme, but also the challenges in using it.

# We start by setting up the CKKS scheme.

parms = EncryptionParameters(scheme_type.ckks)
poly_modulus_degree = 2**13
parms.set_poly_modulus_degree(poly_modulus_degree)
p = [60, 40, 40, 60]  # P_0, P_1, P_2, P_3
parms.set_coeff_modulus(CoeffModulus.Create(poly_modulus_degree, p))

context = SEALContext(parms)
encoder = CKKSEncoder(context)
scale = 2.0 ** 40

keygen = KeyGenerator(context)
secret_key = keygen.secret_key()
public_key = keygen.create_public_key()
relin_keys = keygen.create_relin_keys()

encryptor = Encryptor(context, public_key)
evaluator = Evaluator(context)
decryptor = Decryptor(context, secret_key)

print(f'{encoder.slot_count()}')

# Defining "input" data.
x_vector = [float(i) for i in range(encoder.slot_count())]
x_encoded = encoder.encode(x_vector, scale)
x_1_encrypted = encryptor.encrypt(x_encoded)
print(f'{log2(x_1_encrypted.scale())=}')

# Computations start here.
# encoding constants.
pi_def = 3.14
plain_π = encoder.encode(pi_def, scale)
plain_04 = encoder.encode(0.4, scale)
plain_1 = encoder.encode(1.0, scale)

# To compute x^3 we first compute x^2 and relinearize.
# However, the scale has now grown to 2^80.
print('\nCompute x^2 and relinearize')
x_2_encrypted = evaluator.square(x_1_encrypted)
evaluator.relinearize_inplace(x_2_encrypted, relin_keys)
print(f'{log2(x_2_encrypted.scale())=}')


# Now rescale; in addition to a modulus switch, the scale is reduced down by
# a factor equal to the prime that was switched away (40-bit prime). Hence, the
# new scale should be close to 2^40. Note, however, that the scale is not equal
# to 2^40: this is because the 40-bit prime is only close to 2^40.
print('Rescale x^2')
evaluator.rescale_to_next_inplace(x_2_encrypted)
print(f'{log2(x_2_encrypted.scale())=} After rescale')


# Now x_2_encrypted is at a different level than x_1_encrypted, which prevents us
# from multiplying them to compute x^3. We could simply switch x_1_encrypted to
# the next parameters in the modulus switching chain. However, since we still
# need to multiply the x^3 term with PI (plainCoeff3), we instead compute PI*x
# first and multiply that with x^2 to obtain PI*x^3. To this end, we compute
# PI*x and rescale it back from scale 2^80 to something close to 2^40.

print('\nCompute and rescale π*x')
x_π_encrypted = evaluator.multiply_plain(x_1_encrypted, plain_π)
print(f'{log2(x_π_encrypted.scale())=}')
evaluator.rescale_to_next_inplace(x_π_encrypted)
print(f'{log2(x_π_encrypted.scale())=} after rescale')

# Since x_2_encrypted and x_π_encrypted have the same exact scale and use
# the same encryption parameters, we can multiply them together. We write the
# result to x_2_encrypted, relinearize, and rescale. Note that again the scale
# is something close to 2^40, but not exactly 2^40 due to yet another scaling
# by a prime. We are down to the last level in the modulus switching chain.

print('\nCompute, relinearize, and rescale (π*x)*(x^2).')
x_π_3_encrypted = evaluator.multiply(x_π_encrypted, x_2_encrypted)
evaluator.relinearize_inplace(x_π_3_encrypted, relin_keys)
print(f'{log2(x_π_3_encrypted.scale())=}')
evaluator.rescale_to_next_inplace(x_π_3_encrypted)
print(f'{log2(x_π_3_encrypted.scale())=} After rescale')

#DEBUG
print('\nStart debug')
print('x^2 OK')
p_x2 = decryptor.decrypt(x_2_encrypted)
r_x2 = encoder.decode(p_x2)
print_vector(r_x2, 3, 7)

print('pi*x ok')
p_xpi = decryptor.decrypt(x_π_encrypted)
r_xpi = encoder.decode(p_xpi)
print_vector(r_xpi, 3, 7)

print('pi*x^3, not ok')
p_xpi3 = decryptor.decrypt(x_π_3_encrypted)
r_xpi3 = encoder.decode(p_xpi3)
print_vector(r_xpi3, 3, 7)
print('Stop debug\n')
#DEBUG

# Next we compute the degree one term. All this requires is one MultiplyPlain
# with plainCoeff1. We overwrite x1Encrypted with the result.

print('\nCompute and rescale 0.4*x')
x_04_1_encrypted = evaluator.multiply_plain(x_1_encrypted, plain_04)
print(f'{log2(x_04_1_encrypted.scale())=}')
evaluator.rescale_to_next_inplace(x_04_1_encrypted)
print(f'{log2(x_04_1_encrypted.scale())=} After rescale.')

# Now we would hope to compute the sum of all three terms. However, there is
# a serious problem: the encryption parameters used by all three terms are
# different due to modulus switching from rescaling.

# Encrypted addition and subtraction require that the scales of the inputs are
# the same, and also that the encryption parameters (ParmsId) match. If there
# is a mismatch, Evaluator will throw an exception.
print(f'{context.get_context_data(x_π_3_encrypted.parms_id()).chain_index()=}')
print(f'{context.get_context_data(x_04_1_encrypted.parms_id()).chain_index()=}')
print(f'{context.get_context_data(plain_1.parms_id()).chain_index()=}')

# Let us carefully consider what the scales are at this point. We denote the
# primes in coeff_modulus as P_0, P_1, P_2, P_3, in this order. P_3 is used as
# the special modulus and is not involved in rescalings. After the computations
# above the scales in ciphertexts are:

#     - Product x^2 has scale 2^80 and is at level 2;
#     - Product PI*x has scale 2^80 and is at level 2;
#     - We rescaled both down to scale 2^80/P2 ~= 2^40 and level 1;
#     - Product PI*x^3 has scale (2^80/P_2)^2;
#     - We rescaled it down to scale ((2^80/P_2)^2)/P_1 ~= 2^40 and level 0;
#     - Product 0.4*x has scale 2^80;
#     - We rescaled it down to scale 2^80/P_2 and level 1;
#     - The constant term 1 has scale 2^40 and is at level 2.

# Although the scales of all three terms are approximately 2^40, their exact
# values are different, hence they cannot be added together.

print(f'{log2(x_π_3_encrypted.scale())=}')
print(f'{log2(x_04_1_encrypted.scale())=}')
print(f'{log2(plain_1.scale())=}')

# There are many ways to fix this problem. Since P_2 and P_1 are really close
# to 2^40, we can simply "lie" to Microsoft SEAL and set the scales to be the
# same. For example, changing the scale of PI*x^3 to 2^40 simply means that we
# scale the value of PI*x^3 by 2^120/(P_2^2*P_1), which is very close to 1.
# This should not result in any noticeable error.

# Another option would be to encode 1 with scale 2^80/P_2, do a MultiplyPlain
# with 0.4*x, and finally rescale. In this case we would need to additionally
# make sure to encode 1 with appropriate encryption parameters (ParmsId).

# In this example we will use the first (simplest) approach and simply change
# the scale of PI*x^3 and 0.4*x to 2^40.
print('Normalize scales to 2^40')
x_π_3_encrypted.scale(2**40)
x_04_1_encrypted.scale(2**40)

# We still have a problem with mismatching encryption parameters. This is easy
# to fix by using traditional modulus switching (no rescaling). CKKS supports
# modulus switching just like the BFV scheme, allowing us to switch away parts
# of the coefficient modulus when it is simply not needed.
print('\nNormalize encryption parameters to the lowest level.')
last_parms_id = x_π_3_encrypted.parms_id()
print(f'{last_parms_id=}')
evaluator.mod_switch_to_inplace(x_04_1_encrypted, last_parms_id)
evaluator.mod_switch_to_inplace(plain_1, last_parms_id)

# All three ciphertexts are now compatible and can be added.
print('Compute PI*x^3 + 0.4*x + 1.')
sum_encrypted = evaluator.add(x_04_1_encrypted, x_π_3_encrypted)
sum_encrypted = evaluator.add_plain(sum_encrypted, plain_1)

# First print the true result.
print('expected:')
expected = [pi_def*x**3 + 0.4*x + 1 for x in x_vector]
print_vector(expected, 3, 7)



# We decrypt, decode, and print the result.
print('Actual/FHE calculated:')
plain_result = decryptor.decrypt(sum_encrypted)
result = encoder.decode(plain_result)
print_vector(result, 3, 7)



4096
log2(x_1_encrypted.scale())=40.0

Compute x^2 and relinearize
log2(x_2_encrypted.scale())=80.0
Rescale x^2
log2(x_2_encrypted.scale())=40.00000019347918 After rescale

Compute and rescale π*x
log2(x_π_encrypted.scale())=80.0
log2(x_π_encrypted.scale())=40.00000019347918 after rescale

Compute, relinearize, and rescale (π*x)*(x^2).
log2(x_π_3_encrypted.scale())=80.00000038695836
log2(x_π_3_encrypted.scale())=40.00000135435979 After rescale

Start debug
x^2 OK
[
4.656612873077393e-10	1.0000000018626451	3.999999994877726	8.999999997671694	15.999999993946403	24.999999988358468	36.00000000745058
48.99999998928979	64.0000000121072	80.99999999860302	100.00000002607703	121.0000000083819	143.99999998835847	169.00000000325963
196.00000001536682	224.99999999906868	255.99999999301508	289.00000000651926	323.99999998975545	361.0000000097789	400.0000000037253
]
pi*x ok
[
9.254108590539545e-10	3.1400000006856317	6.279999999051597	9.419999996217484	12.559999996104125	15.699999998235853	18.84000000

### Neste gang
* Teste ut TFHE-rs
* Teste ut 
* Finne ut feilen i koden
* "Play Chess" in FHE with Concrete-ML !5k euro bounty!
* Kanskje flere nye idéer etter eurocrypt

# Takk for meg

# LPSE **L**ightweight **p**assword-**s**trength estimation for password meters

> Password strength can be measured by comparing the
> similarity between a given password vector and a standard
> strong-password vector.
> We determine the similarity between the two password
> vectors from three aspects:
> * the structure of the password
>     * that is, what kinds of characters compose the password and the pro-portions of various characters
> * the password length
> * the number of insertion, substitution, and deletion operations required to transform a given password into a standard strong password. 

### Passordvektor:

En passordvektor ser slik ut: $\alpha = x_1, x_2, x_3, x_4, x_5$. Der verdiene er hennoldsvis vektorverdi av tall, små bokstaver, store bokstaver, spesialtegn og lengden av passordet.

Table 1 – General rules for mapping characters to
vectors.
|Patterns|Vector value| Example char $\rightarrow$ vector|
|--------|------------|------------------------------|
|Digits|1| 8| $\rightarrow$ 1|
|Lowercase letters| 1| d $\rightarrow$ 1|
|Uppercase letters| 2| G $\rightarrow$ 2|
|Special characters| 3| & $\rightarrow$ 3|
|Two identical characters|Equivalent to one character vector|aa $\rightarrow$ 1, 3a3a $\rightarrow$ 2|
|Two consecutive characters|Equivalent to one character vector|AB $\rightarrow$ 2,1a2b $\rightarrow$ 2|

### En sterk passordvektor $\alpha_{s(ecure)}$
Paperet påstår:
> we believe that a strong password should be randomly generated, and the password length should be greater than 16 characters.

Hvor stor vil en passordvektoren til et tilfeldig passord på 16 tegn være?

La oss ta utgangspunkt i et ASCII keyboard med 96 tegn, 10 tall, 26 små og store bokstaver og 32 spesialtegn. Et tilfledig generert passord burde derfor bestå av:
* $16 \times {10 \over 96} \approx 2$ tall.
* $16 \times {26 \over 96} \approx 5$ små bokstaver.
* $16 \times {26 \over 96} \approx 5$ store bokstaver.
* $16 \times {32 \over 96} \approx 6$ spesialtegn.

... og derfor ha passordvektoren $\alpha_s = \{2,5,10,18,18\}$


### Eksempel
Hvilken vektor vil passordet `aa35*TX1` ha?
* Fjern alle like og bokstaver i rekkefølge. `a35*TX1`
* Hvor mange tall har vi? $x_1=3$
* Hvor mange små bokstaver har vi? $x_2=1$
* Hvor mange store bokstaver har vi? $x_3=2\times2=4$
* Hvor mange tegn har vi? $x_4 = 1 \times 3 = 3$
* Hvor langt er passordet? $x_5 = 8$

$\alpha_{\text{aa35*TX1}}=\{3,1,4,3\}$

### Cosine similarity $cos(\phi)$
$$
\begin{align}
\text{Vectors}\;A,B\\
\text{cosine similarity} &= cos(\phi)\\
 &= {{A \cdot B}\over {||A|| \times ||B||}} \\
 &={ {\sum_{n=1}^n A_i \cdot B_i}\over{\sqrt { \sum_{n=1}^n A_i^2} \cdot \sqrt{ \sum_{n=1}^n A_i^2}}}
\end{align}
$$

### Cosine-length similarity $s_c(\alpha)$

$$
\begin{align}
\text{Passord som testes} &= \alpha\\
\text{sikker passordvektor} &= \alpha_s\\
s_c(\alpha) &= \cos(\phi)\times {{\min(|\alpha||\alpha_s|)} \over {\max(|\alpha||\alpha_s|)}} \\
|\alpha| &= \sqrt{\sum{5}_{i=1}(x_i)^2}
\end{align}
$$

### Password-distane
Dersom Microsoft gikk tom for minne for å se på edit-distance ved et 4-alfabets string på lengde over 8, så ville jeg slitt med et 96-alfabets passord på lengde 18.

Flere regler. Dato, brukernavn, plassering på keyboard. Bytte ut leetspeak. Mest vanlige 2 og 3 bokstavskombinasjoner byttes ut. Gjenstår password-length distance.

# HELPSE **H**ormomorphic Encrypted Lighthweight Password-Strength Estimation for password meters

Notater:
ref: https://eprint.iacr.org/2016/421.pdf
